<a href="https://colab.research.google.com/github/moosunny/Graph-Neural-Networks-Practice/blob/main/Chpater%205.%20Including%20Node%20Features%20with%20Vanila%20Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Chpater 5. Including Node Features with Vanila Neural Networks

이전 Chapter에서는 단순 노드간 연결 관계를 나타내는 topology(네트워크 요소) 만을 취급했지만 그래프 데이터셋에서의 노드와 엣지는 점수, 색상, 단어 등을 나타내는 Feature representing을 가능케 한다.

노드와 엣지 feature는 정형 데이터와도 동일한 구조를 갖기 때문에, 신경망 구조를 표현할 수 있다. 이번 Chapter에서는 Cora와 Facebook 데이터를 통해 Pytorch 기반의 Vanila Nueral Network 작동하는 방식을 학습한다.

이전에 활용한 Zachary 카라테 클럽 데이터셋과 가르게 Cora와 Face book 데이터셋에는 새로운 유형의 노드 정보가 담겨있다. 해당 두 데이터 셋의 경우 사용자의 나이, 성별, 관심사와 같은 노드에 대한 정보가 있다.

따라서, 이번에는 간단한 그래프 신경망 모델을 통해 Node Classification 문제를 해결한다.

In [4]:
import torch

torch.__version__

'2.5.1+cu124'

In [8]:
# Colab 환경에 맞는 pytorch geometric cuda 버전 설치
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.5.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00


In [32]:
# Metric 정의
def accuracy(y_pred, y_true):
  return torch.sum(y_pred == y_true) / len(y_true)

In [33]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

# Multi Layer Perceptron 모델
class MLP(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.linear1 = Linear(dim_in, dim_h)
    self.linear2 = Linear(dim_h, dim_out)

  def forward(self, x):
    x = F.relu(self.linear1(x))
    x = self.linear2(x)

    return F.log_softmax(x, dim=1)

  def fit(self, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)

    self.train()
    for epoch in range(epochs + 1):
      optimizer.zero_grad()
      out = self(data.x)
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      acc = accuracy(out[data.train_mask]. argmax(dim=1), data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      if epoch % 20 == 0:
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask]. argmax(dim=1), data.y[data.val_mask])
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

  def test(self, data):
    self.eval()
    out = self(data.x)
    acc = accuracy(out.argmax(dim=1)[data.test_mask],
    data.y[data.test_mask])

    return acc

In [28]:
from torch_geometric.datasets import Planetoid

# Cora 데이터셋
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

print(f'Dataset: {dataset}')
print('---------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')

Dataset: Cora()
---------------
Number of graphs: 1
Number of nodes: 2708


In [29]:
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Number of features: 1433
Number of classes: 7


In [30]:
print(f'Graph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [37]:
# Face book 페이지 데이터
from torch_geometric.datasets import FacebookPagePage

dataset = FacebookPagePage(root=".")
data = dataset[0]

print(f'Dataset: {dataset}')
print('-----------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: FacebookPagePage()
-----------------------
Number of graphs: 1
Number of nodes: 22470
Number of features: 128
Number of classes: 4

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: True


In [31]:
import pandas as pd

df_x = pd.DataFrame(data.x.numpy())
df_x['label'] = pd.DataFrame(data.y)

df_x

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2704,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [34]:
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)


In [35]:
mlp.fit(data, epochs=100)

Epoch   0 | Train Loss: 1.951 | Train Acc: 12.86% | Val Loss: 1.96 | Val Acc: 13.40%
Epoch  20 | Train Loss: 0.135 | Train Acc: 100.00% | Val Loss: 1.54 | Val Acc: 45.00%
Epoch  40 | Train Loss: 0.014 | Train Acc: 100.00% | Val Loss: 1.63 | Val Acc: 46.40%
Epoch  60 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 1.57 | Val Acc: 48.80%
Epoch  80 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 1.47 | Val Acc: 52.00%
Epoch 100 | Train Loss: 0.009 | Train Acc: 100.00% | Val Loss: 1.43 | Val Acc: 53.00%


In [36]:
acc = mlp.test(data)
print(f'MLP test accuracy: {acc*100:.2f}%')

MLP test accuracy: 53.80%
